In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import matplotlib
matplotlib.font_manager._rebuild()
plt.rcParams['font.family'] = 'Times New Roman' #全体のフォントを設定
%matplotlib inline

/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [5]:
import sys
sys.path.append("/notebooks/nb_IEEE/opmap/")

In [6]:
import opmap
from opmap.rawCam import RawCam
from opmap.vmemMap import VmemMap
from opmap.apdMap import APDMap
from opmap.phaseMapHilbert import PhaseMapHilbert as PhaseMap
from opmap.phaseVarianceMap import PhaseVarianceMap

In [7]:
def FFT(vmem_1d, N, dt, fc_low=None, fc_high=None):
    fq = np.linspace(0, 1.0/dt, N)
    F = np.fft.fft(vmem_1d)
    if fc_low is not None:
        F[(fq <= fc_low)] = 0
    if fc_high is not None:
        F[(fq >= fc_high)] = 0
    
    F_abs = np.abs(F)
    F_abs_amp = F_abs/N
#     F_abs_amp[0] = F_abs_amp[0]/2
    F_abs_amp[0] = 0
    
    F_ifft = np.fft.ifft(F)
    F_ifft_real = F_ifft.real * 2
    
    return F_ifft_real

In [8]:
def isochronal(pmap, threshold):
    contour = np.zeros_like(pmap.data)

    for i in range(contour.shape[1]):
        for j in range(contour.shape[2]):        
            if pmap.roi[i,j] == 1:
                counter = 0
                for time in range(contour.shape[0]):
                    if time == 0:
                        pass
                    elif ((pmap.data[time-1, i, j] < threshold) * (pmap.data[time, i, j] > threshold)) == 1:
                        counter = 0

                    contour[time, i, j] = counter
                    counter += 1
                
    return contour

## 冷却実験のデータ総解析

In [9]:
data_dir = ['20181218-1/C001H001S0007/', '20181218-1/C001H001S0008/', '20181218-1/C001H001S0009/',
            '20181218-1/C001H001S0011/', '20181218-1/C001H001S0017/', '20181218-1/C001H001S0024/',
            '20181220-1/C001H001S0007/', '20181220-1/C001H001S0009/', '20181220-1/C001H001S0010/',
            '20181220-1/C001H001S0014/', '20181220-1/C001H001S0015/', '20181220-1/C001H001S0022/']

In [ ]:
parent_path = '/mnt/Jalife/ExperimentData/'
for folder in data_dir:
    data_path = os.path.join(parent_path, folder)
    print(data_path)
    start = 29000
    end = 30000
    date = data_path.split('/')[-3]
    session_name = data_path.split('/')[-2]
    save_parent = os.path.join('/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/', '%s/%s/%s-%s' % (date, session_name, start, end))
    
    raw = RawCam(path=data_path, cam_type='sa4',
              image_width=256, image_height=256, frame_start=start, frame_end=end)
    
    vmem = VmemMap(raw)
    vmem.setDiffRange(diff_min=20)
    vmem.morphROI(closing=4)
    vmem.morphROI(erosion=8)

    for i in range(vmem.data.shape[1]):
        for j in range(vmem.data.shape[2]):
            vmem.data[:, i, j] = FFT(vmem.data[:, i, j], vmem.data.shape[0], 0.002, fc_low=2, fc_high=50)
    vmem.smooth(size=9)
    
    pmap = PhaseMap(vmem)

    if not os.path.exists(os.path.join(save_parent, 'pmap')):
        os.makedirs(os.path.join(save_parent, 'pmap'))
    
    pmap.saveImage(os.path.join(save_parent, 'pmap'), img_type='png', skip=1)
    cmd = 'ffmpeg -r 30 -y -i "{0}/pmap/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "{0}/pmap_movie.mp4"'.format(save_parent)
    subprocess.call(cmd, shell=True)

/mnt/Jalife/ExperimentData/20181218-1/C001H001S0007/
